In [11]:
!pip install pypdf

In [45]:
!pip install langchain-google-genai

In [46]:
!pip install langchain_community

In [47]:
!pip install -U langchain-text-splitters

In [23]:
!pip install chromadb

     -------------------------------------- 526.8/526.8 kB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 47.2/47.2 kB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 91.9/91.9 kB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 60.1/60.1 kB 3.3 MB/s eta 0:00:00
     -------------------------------------- 158.3/158.3 kB 9.9 MB/s eta 0:00:00
     -------------------------------------- 106.1/106.1 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 5.4 MB/s eta 0:00:00
     -------------------------------------- 150.6/150.6 kB 4.4 MB/s eta 0:00:00
     ---------------------------------------- 78.3/78.3 kB 4.5 MB/s eta 0:00:00
     ---------------------------------------- 60.8/60.8 kB ? eta 0:00:00
     ---------------------------------------- 67.3/67.3 kB ? eta 0:00:00
  Installing build dependencies: started
  Installing buil

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.24.0 requires tokenizers!=0.11.3,<0.14,>=0.11.1, but you have tokenizers 0.19.1 which is incompatible.


In [12]:
from langchain_community.document_loaders import PyPDFLoader

In [13]:
content = PyPDFLoader(r'C:\Users\Asus\Downloads\2404.07143.pdf')
pages=content.load_and_split()

In [14]:
from langchain_text_splitters import NLTKTextSplitter
text_splitter = NLTKTextSplitter(chunk_size=500,chunk_overlap=100)
chunks=text_splitter.split_documents(pages)
print(len(chunks))
print(type(chunks[0]))

Created a chunk of size 568, which is longer than the specified 500
Created a chunk of size 506, which is longer than the specified 500
Created a chunk of size 633, which is longer than the specified 500


110
<class 'langchain_core.documents.base.Document'>


In [15]:
chunks[0]

Document(page_content='Preprint.\n\nUnder review.\n\nLeave No Context Behind:\nEfficient Infinite Context Transformers with Infini-attention\nTsendsuren Munkhdalai, Manaal Faruqui and Siddharth Gopal\nGoogle\ntsendsuren@google.com\nAbstract\nThis work introduces an efficient method to scale Transformer-based Large\nLanguage Models (LLMs) to infinitely long inputs with bounded memory\nand computation.\n\nA key component in our proposed approach is a new at-\ntention technique dubbed Infini-attention.', metadata={'source': 'C:\\Users\\Asus\\Downloads\\2404.07143.pdf', 'page': 0})

In [16]:
f = open('googleapi.txt')

In [17]:
Google_api_key = f.read()

In [26]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embedding_model = GoogleGenerativeAIEmbeddings(google_api_key=Google_api_key, model="models/embedding-001")

In [27]:
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(chunks, embedding_model, persist_directory=r"C:\Users\Asus\leave\chroma_db_")
db.persist()

In [29]:
db_connection = Chroma(persist_directory="./chroma_db_", embedding_function=embedding_model)

In [30]:
retriever = db_connection.as_retriever(search_kwargs={"k": 5})
print(type(retriever))

<class 'langchain_core.vectorstores.VectorStoreRetriever'>


In [31]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [32]:
chat_template = ChatPromptTemplate.from_messages([
    # System Message Prompt Template
    SystemMessage(content="""You are a Helpful AI Bot. 
    Your task is to provide assistance based on the context given by the user. 
    Make sure your answers are relevant and helpful."""),
    # Human Message Prompt Template
    HumanMessagePromptTemplate.from_template("""Answer the question based on the given context.
    Context:
    {context}
    
    Question: 
    {question}
    
    Answer: """)])

In [33]:
from langchain_google_genai import ChatGoogleGenerativeAI
chat_model = ChatGoogleGenerativeAI(google_api_key=Google_api_key, model="gemini-1.5-pro-latest")

In [34]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [35]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | chat_template
    | chat_model
    | output_parser)

In [41]:
from IPython.display import Markdown as md

In [39]:
response = rag_chain.invoke("Can you please tell me about meory retival and memory update")
response

"## Memory Retrieval and Update in the Context of Efficient Attention Mechanisms:\n\nThe provided text discusses approaches to improve the efficiency of attention mechanisms in handling long sequences, focusing on memory retrieval and update processes. Let's break down the key points:\n\n**Challenges with Standard Attention:**\n\n* **Growing Memory Consumption:** Standard attention mechanisms require storing key and value (KV) pairs for each element in the input sequence. This leads to increased memory usage as the sequence length grows, making it impractical for long sequences.\n\n**Solutions - Compressive Memory and Retrieval:**\n\n* **Compressive Memory:** Instead of storing all KV pairs, a compressive memory approach maintains a fixed number of parameters. It efficiently stores information by updating these parameters in a way that allows for later retrieval. \n* **Retrieval Mechanisms:** When processing new sequences, the attention query interacts with the compressed memory to ret

In [40]:
md(response)

## Memory Retrieval and Update in the Context of Efficient Attention Mechanisms:

The provided text discusses approaches to improve the efficiency of attention mechanisms in handling long sequences, focusing on memory retrieval and update processes. Let's break down the key points:

**Challenges with Standard Attention:**

* **Growing Memory Consumption:** Standard attention mechanisms require storing key and value (KV) pairs for each element in the input sequence. This leads to increased memory usage as the sequence length grows, making it impractical for long sequences.

**Solutions - Compressive Memory and Retrieval:**

* **Compressive Memory:** Instead of storing all KV pairs, a compressive memory approach maintains a fixed number of parameters. It efficiently stores information by updating these parameters in a way that allows for later retrieval. 
* **Retrieval Mechanisms:** When processing new sequences, the attention query interacts with the compressed memory to retrieve relevant information. This information is then used to compute attention scores and update the model's representation.

**Specific Techniques Mentioned:**

* **Katharopoulos et al. (2020):** The text highlights the adoption of an update rule and retrieval mechanism proposed by Katharopoulos et al. due to its simplicity and effectiveness. 
* **Infini-attention:** This approach reuses key, value, and query states from standard attention computations for long-term memory consolidation. Instead of discarding old KV states, they are stored in the compressive memory and retrieved later using attention query states when processing subsequent sequences.

**Benefits:**

* **Bounded Storage and Computation:** By using a fixed-size memory, compressive memory approaches avoid the memory limitations of standard attention, enabling efficient processing of long sequences.
* **Efficient Retrieval:** Retrieval mechanisms allow relevant information to be accessed quickly and incorporated into the model's representation, contributing to better performance. 

**Additional Considerations:**

* **System-level Optimizations:** The text mentions exploring hardware-specific optimizations to further improve the efficiency of attention computations.

**In summary, memory retrieval and update techniques play a crucial role in enhancing the efficiency and effectiveness of attention mechanisms for handling long sequences. By employing compressive memory and efficient retrieval methods, models can maintain performance while managing memory constraints.** 
